In [1]:
import pandas as pd
import numpy as np
from scipy import  stats
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csc_matrix, hstack
from sklearn.model_selection import cross_validate, train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin, clone
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import pickle
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict
from matplotlib_venn import venn2
import time

In [ ]:
class FeatureSelector():
    def __init__(self, estimator,
                       metric,\
                       larger_is_better,\
                       cv,
                       use_values,\
                       use_recursion,
                       fill_na,\
                       show_progress, 
                       early_stopping = None):
        '''
        Инициализирует модель для отбора признаков
        
        Параметры:
            1) estimator - модель
            2) metric - метрика качества (названия метрик sklearn + может быть кастомная)
            3) larger_is_better - критерий оптимизации (чем больше, тем лучше)
            4) cv - схема валидации
            5) use_values - индексы столбцов, в которых требуется отобрать значения
            6) use_recursion - использовать рекурсию в отборе
            7) fill_na - значение, которым заполняются np.nan
            8) show_progress - печатать результаты валидации
            9) early_stopping - число итераций без улучшения метрики для ранней остановки отбора
        Возвращает:
            1) fit - производит отбор признаков
            2) transform - оставляет отобранные признаки
            3) return_self - возвращает 
                - best_features - отобранные признаки(список)
                - D_best_features - отобранные значения признаков (словарь: {признак:значения})
                - best_score - лучшее значение метрики
        '''
        self.estimator = estimator
        self.metric = metric
        self.cv = cv
        self.use_values = use_values        
        self.use_recursion = use_recursion
        self.show_progress = show_progress
        self.early_stopping = early_stopping
        self.fill_na = fill_na
        self.larger_is_better = larger_is_better
    def fit(self, X, Y):        
        flag = isinstance(X[:, 0], csc_matrix)
        # список с результатами валидации
        column_value_score = []
        # проходим по признакам
        for i in tqdm_notebook(range(X.shape[1])):
            # если формат матрицы признаков == csc_matrix
            if flag:
                # выбираем столбец, преобразуем
                ser = pd.DataFrame(X[:, i].todense())[0].values.flatten()
            # если формат != csc_matrix
            else:
                # выбираем столбец
                ser = X[:, i]        
            # если столбец в списке с проверкой значений 
            if self.use_values is not None:                
                if i in self.use_values:
                    # уникальные значения столбца
                    unique_values = np.unique(ser)  
                    # валидируем каждое значение
                    for val in unique_values:
                        _x = np.int32(ser==val).reshape(-1,1)
                        column_value_score.append((i, val,\
                                                   cross_val_score(self.estimator,\
                                                                   _x, Y,\
                                                                   scoring = self.metric,\
                                                                   cv = self.cv).mean()))
                else: 
                    # валидируем столбец
                    column_value_score.append((i, None,\
                                               cross_val_score(self.estimator,\
                                                               _x, Y,\
                                                               scoring = self.metric,\
                                                               cv = self.cv).mean()))
            else:
                # валидируем столбец
                    column_value_score.append((i, None,\
                                               cross_val_score(self.estimator,\
                                                               ser.reshape(-1,1), Y,\
                                                               scoring = self.metric,\
                                                               cv = self.cv).mean()))
                

        # признаки и значения признаков в порядке убывания валидации
        order = np.array(sorted(column_value_score, key = lambda x: x[-1], reverse = True))[:, :2]             
        # список лучших признаков
        best_features = []
        # словарь лучших значений признаков
        D_best_features = defaultdict(list)
        # список с признаками, не давшими прироста
        to_drop = []
        
        # лучшее значение метрики
        if self.larger_is_better:
            best_score = 0
        else:
            best_score = np.inf            
        counter = 0
        # проходим по признакам и значениям признаков в порядке убывания валидации
        for feature, feature_value in tqdm_notebook(order):   

            # добавляем текущие признаки/значения
            if feature_value is None:
                best_features.append(feature)               
            else:
                D_best_features[feature].append(feature_value)

            # обновляем матрицы
            L = []
            for k, v in D_best_features.items():
                if isinstance(X[:, k], csc_matrix):
                    L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))
                else:
                    L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))

            if flag:
                if (len(best_features)>0) & (len(L)>0):
                    _X = csc_matrix(hstack([X[:, best_features], csc_matrix(np.column_stack(L)) ]))
                elif (len(best_features)==0) & (len(L)>0):
                    _X = csc_matrix(np.column_stack(L))
                elif (len(best_features)>0) & (len(L)==0):
                    _X = csc_matrix(X[:, best_features])                    
                        
            else:
                if (len(best_features)>0) & (len(L)>0):
                    _X = np.column_stack([X[:, best_features], np.column_stack(L)])
                elif (len(best_features)==0) & (len(L)>0):
                    _X = np.column_stack(L)
                elif (len(best_features)>0) & (len(L)==0):
                    _X = X[:, best_features] 
            # считаем валидацию    
            current_score = cross_val_score(self.estimator, _X, Y, scoring = self.metric, cv = self.cv).mean()
            # если метрика улучшилась
            if self.larger_is_better:
                if current_score>best_score:
                    # обновляем лучшую метрику
                    best_score = current_score
                    counter = 0
                    # печатаем 
                    if self.show_progress:
                        print('new best_score = {}'.format(best_score))
                # если метрика не улучшилась
                else: 
                    counter+=1
                    # удаляем признак/значение
                    if feature_value is None:
                        best_features = [val for val in best_features if val != feature]
                        to_drop.append((feature, None))
                    else:
                        D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                        to_drop.append((feature, feature_value))
                    if counter == self.early_stopping:
                        break
            else:
                if current_score<best_score:
                    # обновляем лучшую метрику
                    best_score = current_score
                    counter = 0
                    # печатаем 
                    if self.show_progress:
                        print('new best_score = {}'.format(best_score))
                    # если метрика не улучшилась
                else: 
                    counter+=1
                    # удаляем признак/значение
                    if feature_value is None:
                        best_features = [val for val in best_features if val != feature]
                        to_drop.append((feature, None))
                    else:
                        D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                        to_drop.append((feature, feature_value))
                    if counter == self.early_stopping:
                        break

        if self.use_recursion:
            # запускаем бесконечный цикл
            while True:
                # списки лучших признаков до и после
                to_drop_before = to_drop
                to_drop_after = []
                # проходим по признакам и значениям признаков в порядке убывания валидации
                for feature, feature_value in tqdm_notebook(to_drop_before):   
                    # добавляем текущие признаки/значения
                    if feature_value is None:
                        best_features.append(feature)               
                    else:
                        D_best_features[feature].append(feature_value)

                    # обновляем матрицы
                    L = []
                    for k, v in D_best_features.items():
                        if isinstance(X[:, k], csc_matrix):
                            L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))
                        else:
                            L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))

                    if flag:
                        if (len(best_features)>0) & (len(L)>0):
                            _X = csc_matrix(hstack([X[:, best_features], csc_matrix(np.column_stack(L)) ]))
                        elif (len(best_features)==0) & (len(L)>0):
                            _X = csc_matrix(np.column_stack(L))
                        elif (len(best_features)>0) & (len(L)==0):
                            _X = csc_matrix(X[:, best_features])                    

                    else:
                        if (len(best_features)>0) & (len(L)>0):
                            _X = np.column_stack([X[:, best_features], np.column_stack(L)])
                        elif (len(best_features)==0) & (len(L)>0):
                            _X = np.column_stack(L)
                        elif (len(best_features)>0) & (len(L)==0):
                            _X = X[:, best_features] 

                    # считаем валидацию    
                    current_score = cross_val_score(self.estimator, _X, Y, scoring = self.metric, cv = self.cv).mean()
                    
                    
                    
                    
                    
                    
                    if self.larger_is_better:
                        if current_score>best_score:
                            # обновляем лучшую метрику
                            best_score = current_score
                            counter = 0
                            # печатаем 
                            if self.show_progress:
                                print('new best_score = {}'.format(best_score))
                            # если метрика не улучшилась
                        else: 
                            # удаляем признак/значение
                            if feature_value is None:
                                best_features = [val for val in best_features if val != feature]
                                to_drop_after.append((feature, None))
                            else:
                                D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                                to_drop_after.append((feature, feature_value))
                    else:
                        if current_score<best_score:
                            # обновляем лучшую метрику
                            best_score = current_score
                            counter = 0
                            # печатаем 
                            if self.show_progress:
                                print('new best_score = {}'.format(best_score))
                        else: 
                            # удаляем признак/значение
                            if feature_value is None:
                                best_features = [val for val in best_features if val != feature]
                                to_drop_after.append((feature, None))
                            else:
                                D_best_features[feature] = [val for val in D_best_features[feature] if val != feature_value]    
                                to_drop_after.append((feature, feature_value))
                    

                # если списки одинаковые, останавливаем отбор
                if len(to_drop_after) == len(to_drop_before):
                    break
                # если разные - обновляем списки до и после
                else:
                    to_drop_before = to_drop_after
                    to_drop_after = []
                    
        self.best_features = best_features
        self.D_best_features = D_best_features
        self.best_score =best_score
        self.flag = flag
    def transform(self, X):
              
        if len(self.best_features) !=0:
            x1 = X[:, self.best_features]
        else:
            x1 = None
        if len(list(self.D_best_features.keys())) !=0:
            L=[]
            for k, v in self.D_best_features.items():
                if self.flag:
                    L.append(pd.DataFrame(X[:, k].tocsc().todense())[0].apply(lambda x: x if x in v else self.fill_na))                    
                else:
                    L.append(pd.Series(X[:, k].flatten()).apply(lambda x: x if x in v else self.fill_na))
            x2 = np.column_stack(L)
        else:
            x2 = None
            
        if (x1 is not None) & (x2 is not None):
            if self.flag: 
                _X = csc_matrix(hstack([x1, x2]))
            else:
                _X = np.column_stack([x1, x2])
                
        if (x1 is not None) & (x2 is None):
            _X = x1
        if (x1 is None) & (x2 is not None):
            if self.flag:
                _X = csc_matrix(x2)
            else:
                _X = x2
        return _X     
        
    def return_self(self):
        return self


class StackingRegressor():
    def __init__(self, models, cv_strat):
        '''
        models - список с ансамблем моделей
        nfolds - число фолдов для ооф предсказаний
        seed - генератор случайных чисел
        '''        
        self.models = models
        self.cv_strat = cv_strat        
    def fit(self, X, y):
        '''
        1) обучаем модели на валидации
        2) сохраняем обученные модели
        '''        
        try:
            _X, _y = np.array(X), np.array(y)
        except:
            _X, _y = X, y        
        estimators = []
        for model in tqdm_notebook(self.models):   
            try:
                for tr_idx, val_idx in self.cv_strat.split(_X):
                    model.fit(_X[tr_idx], _y[tr_idx])
                    estimators.append(model)
            except:
                for tr_idx, val_idx in self.cv_strat.split(_X, _y):
                    model.fit(_X[tr_idx], _y[tr_idx])
                    estimators.append(model)
                    
        self.fitted_estimators = estimators
    def get_metafeatures(self, X):
        '''
        с помощью обученных моделей получаем метапризнаки
        '''
        try:
            _X = np.array(X)
        except:
            _X = X
        L = []    
        for estimator in self.fitted_estimators:
            L.append(estimator.predict(_X))
        return np.column_stack(L)

In [3]:
times = ['time%d' % i for i in range(1, 11)]
sites = ['site%d' % i for i in range(1, 11)]

df_tr = pd.read_csv('train_sessions.csv', parse_dates = times).set_index('session_id').sort_values('time1')
df_te = pd.read_csv('test_sessions.csv', parse_dates = times).set_index('session_id')
y_tr = df_tr['target']

In [4]:
SEED = 13
FILL_NA = -999
LEN = len(df_tr)
IDX_SPLIT = np.int32(np.around(LEN*.9))

In [5]:
# оставляем сайты, которые посещались Элис + присутствуют в тесте
sites_to_use = np.intersect1d(pd.Series(df_tr[df_tr['target'] == 1][sites].values.flatten()).dropna().unique(),\
                              pd.Series(df_te[sites].values.flatten()).dropna().unique())

# трейн
sites_tr = df_tr[sites].applymap(lambda x: x if x in sites_to_use else np.nan)
# тест
sites_te = df_te[sites]
# трейн + тест
sites_full = pd.concat([sites_tr, sites_te], 0)
# переводим в строку
sites_full_str = sites_full.fillna(FILL_NA).astype(str).apply(lambda row:'_'.join(row), axis = 1)

# дата начала сессии
start_tr = df_tr[times].min(1)
start_te = df_te[times].min(1)
ts_columns = ['день_года', 'неделя_года', 'год', 'меяц',\
              'день', 'день_недели', 'час', 'минута1','минута2', 'секунда1', 'секунда2']

ts_D = {}
for name, ser in zip(('tr', 'te'), (start_tr, start_te)):
    _df = pd.concat([ser.dt.dayofyear,\
                     ser.dt.weekofyear,\
                     ser.dt.year,\
                     ser.dt.month,\
                     ser.dt.day,\
                     ser.dt.dayofweek,\
                     ser.dt.hour,\
                     ser.dt.minute//60,\
                     ser.dt.minute%60,\
                     ser.dt.second//60,\
                     ser.dt.second%60], 1)
    _df.columns = ts_columns    
    ts_D[name] = _df

# оставляем значения из теста
for col in ts_columns:
    to_use = ts_D['te'][col].dropna().unique()
    ts_D['tr'][col] = ts_D['tr'][col].apply(lambda x: x if x in to_use else np.nan)

In [6]:
# ohe
encoder = OneHotEncoder(handle_unknown='ignore')
_X_ts_ohe_tr = csc_matrix(encoder.fit_transform(ts_D['tr'].fillna(FILL_NA)))
_X_ts_ohe_te = csc_matrix(encoder.transform(ts_D['te'].fillna(FILL_NA)))

# нормализуем 
mnmx_scaler = MinMaxScaler()
mnmx_scaler.fit(ts_D['tr'].fillna(FILL_NA))
_X_ts_tr = mnmx_scaler.transform(ts_D['tr'].fillna(FILL_NA))
_X_ts_te = mnmx_scaler.transform(ts_D['te'].fillna(FILL_NA))

In [7]:
def _csc_hstack_arrays(arrays):
    ''' объединяет массивы типа csc_matrix (по колонкам)'''
    return csc_matrix(hstack(arrays))

In [8]:
def _prepare_data(sites_full_str, \
                  _X_ts_ohe_tr, _X_ts_ohe_te,\
                  _X_ts_tr, _X_ts_te,\
                  y_tr, vec):
    ''' 
    готовит наборы данных
    1) трейн(ТРЕЙН+ОТЛОЖЕННАЯ), тест
    2) для логита, для бустинга
        
    '''
    _y_TR, _y_HOLD = np.array(y_tr)[:IDX_SPLIT], np.array(y_tr)[IDX_SPLIT:]
    
    # tfidf    
    tfidf_full = csc_matrix(vec.fit_transform(sites_full_str))
    tfidf_tr, tfidf_te = tfidf_full[:LEN], tfidf_full[LEN:]
        
    _X_logit_tr = _csc_hstack_arrays([_X_ts_ohe_tr, tfidf_tr])
    _X_logit_TR, _X_logit_HOLD = _X_logit_tr[:IDX_SPLIT], _X_logit_tr[IDX_SPLIT:]
    _X_logit_te = _csc_hstack_arrays([_X_ts_ohe_te, tfidf_te])
    
    _X_lgb_tr = _csc_hstack_arrays([_X_ts_tr, tfidf_tr])
    _X_lgb_TR, _X_lgb_HOLD = _X_lgb_tr[:IDX_SPLIT], _X_lgb_tr[IDX_SPLIT:]
    _X_lgb_te = _csc_hstack_arrays([_X_ts_te, tfidf_te])
        
       
    values = (_X_logit_tr, _X_logit_TR, _X_logit_HOLD, _X_logit_te,\
              _X_lgb_tr, _X_lgb_TR, _X_lgb_HOLD, _X_lgb_te,_y_TR, _y_HOLD)
    names = ['logit_tr', 'logit_TR', 'logit_HOLD', 'logit_te',\
             'lgb_tr', 'lgb_TR', 'lgb_HOLD', 'lgb_te', 'y_TR', 'y_HOLD']
    
    return dict(zip(names, values))

In [9]:
# модели
logit_clf = LogisticRegression(random_state = SEED)
lgb_clf = LGBMClassifier(random_state = SEED)

_ngram_ranges_L = ((1, 1), (1, 2), (1, 3))
_max_features_L = (10, 100)
_nfolds_L = [3, 4, 5]

In [11]:
start = time.time()

L_selection = []
L_scores = []
# используем разное число фолдов
for _nfolds in tqdm_notebook(_nfolds_L):
    
    # схема валидации
    cv_strat = TimeSeriesSplit(_nfolds)
    
    # инициализируем модели отбора признаков
    lgb_selector = FeatureSelector(estimator = lgb_clf,\
                                               metric = 'roc_auc',\
                                               larger_is_better = True,\
                                               cv = cv_strat,\
                                               use_values = np.arange(_X_ts_tr.shape[1]),\
                                               use_recursion = False,\
                                               fill_na = FILL_NA,\
                                               show_progress = False)
    logit_selector = FeatureSelector(estimator = logit_clf,\
                                                 metric = 'roc_auc',\
                                                 larger_is_better = True,\
                                                 cv = cv_strat,\
                                                 use_values = None,\
                                                 use_recursion = False,\
                                                 fill_na = FILL_NA,\
                                                 show_progress = False)
    
    # задаем tfidf vectorizer 
    for _ngram_range in tqdm_notebook(_ngram_ranges_L):
        for _max_features in tqdm_notebook(_max_features_L):

            # получаем датасеты
            D_datasets = _prepare_data(sites_full_str,\
                                        _X_ts_ohe_tr, _X_ts_ohe_te,\
                                        _X_ts_tr, _X_ts_te,\
                                        y_tr = y_tr,\
                                        vec = TfidfVectorizer(ngram_range = _ngram_range, max_features = _max_features))

            # отбираем признаки
            logit_selector.fit(D_datasets['logit_TR'], D_datasets['y_TR'])        
            x1_TR = logit_selector.transform(D_datasets['logit_TR'])
            x1_HOLD = logit_selector.transform(D_datasets['logit_HOLD'])
            x1_tr = logit_selector.transform(D_datasets['logit_tr'])
            x1_te = logit_selector.transform(D_datasets['logit_te'])
            
            # тестируем на отложенной
            logit_clf.fit(x1_TR, D_datasets['y_TR'])
            auc_logit_hold = roc_auc_score(D_datasets['y_HOLD'], logit_clf.predict_proba(x1_HOLD)[:, 1])

            # отбираем признаки
            lgb_selector.fit(D_datasets['lgb_TR'], D_datasets['y_TR'])        
            x2_TR = lgb_selector.transform(D_datasets['lgb_TR'])
            x2_HOLD = lgb_selector.transform(D_datasets['lgb_HOLD'])
            x2_tr = lgb_selector.transform(D_datasets['lgb_tr'])
            x2_te = lgb_selector.transform(D_datasets['lgb_te'])
            
            # тестируем на отложенной
            lgb_clf.fit(x2_TR, D_datasets['y_TR'])
            auc_lgb_hold = roc_auc_score(D_datasets['y_HOLD'], lgb_clf.predict_proba(x2_HOLD)[:, 1])

            # сохраняем результаты
            L_scores.append((_nfolds, _max_features,\
                             logit_selector.best_score, lgb_selector.best_score,\
                             auc_logit_hold, auc_lgb_hold))
            print('nfolds = {}, logit hold = {:.5f}, lgb hold = {:.5f}'.format(_nfolds, auc_logit_hold, auc_lgb_hold))
            
            # сохраняем число фолдов, ngram_range, max_featuers, результаты отбора признаков, словарь с датасетами
            L_selection.append(((x1_tr, x1_te), (x2_tr, x2_te)))

end = time.time()
duration = (end - start) // 60
print('выполнено за {} часов, {} минут'.format(duration//60, duration%60))


nfolds = 3, logit hold = 0.94803, lgb hold = 0.97760



nfolds = 3, logit hold = 0.95267, lgb hold = 0.98340




nfolds = 3, logit hold = 0.94683, lgb hold = 0.98233



nfolds = 3, logit hold = 0.95181, lgb hold = 0.97931




nfolds = 3, logit hold = 0.94683, lgb hold = 0.98634


...


nfolds = 5, logit hold = 0.96189, lgb hold = 0.97060




nfolds = 5, logit hold = 0.95502, lgb hold = 0.97330



nfolds = 5, logit hold = 0.97178, lgb hold = 0.97697




nfolds = 5, logit hold = 0.95933, lgb hold = 0.97330



nfolds = 5, logit hold = 0.97211, lgb hold = 0.97519



выполнено за 7.0 часов, 34.0 минут


In [11]:
report_df= pd.read_pickle('report.pkl')
report_df['mean_lgb'] = report_df.iloc[:, -2:].mean(1)
report_df['mean_logit'] = report_df.iloc[:, -4:-2].mean(1)
report_df.loc[(report_df['mean_lgb'] - report_df['mean_logit']).abs().sort_values().index]

,nfolds,tfidf_(maxfeatures),tfidf_(ngram_range),logit_best_cv,logit_hold,lgb_best_cv,lgb_hold,mean_lgb,mean_logit
17,5,100,"(1, 3)",0.958845,0.972107,0.954015,0.975193,0.964604,0.963061
15,5,100,"(1, 2)",0.958924,0.971781,0.950195,0.976971,0.963583,0.960988
13,5,100,"(1, 1)",0.955603,0.961886,0.951838,0.970604,0.961221,0.956862
16,5,10,"(1, 3)",0.954831,0.959330,0.951156,0.973302,0.962229,0.955243
14,5,10,"(1, 2)",0.955756,0.955018,0.955072,0.973300,0.964186,0.955045
12,5,10,"(1, 1)",0.953778,0.957055,0.955421,0.981671,0.968546,0.956238
3,3,100,"(1, 2)",0.969523,0.951814,0.944837,0.979313,0.962075,0.948325
5,3,100,"(1, 3)",0.969154,0.951774,0.944665,0.979904,0.962284,0.948220
0,3,10,"(1, 1)",0.963315,0.948033,0.933474,0.977598,0.955536,0.940754
1,3,100,"(1, 1)",0.965393,0.952674,0.943302,0.983397,0.963350,0.947988


* выбрали оптимальную валидацию (по разнице валидационного и отложенного скора)(TimeSeriesSplit(5))
* выбираем параметры tfidf
* max_features = 100 (пока оставим так)

In [12]:
tscv5 = TimeSeriesSplit(5)
# инициализируем модели отбора признаков
lgb_selector = FeatureSelector(estimator = lgb_clf,\
                               metric = 'roc_auc',\
                               larger_is_better = True,\
                               cv = tscv5,\
                               use_values = np.arange(_X_ts_tr.shape[1]),\
                               use_recursion = False,\
                               fill_na = FILL_NA,\
                               show_progress = False)
logit_selector = FeatureSelector(estimator = logit_clf,\
                                 metric = 'roc_auc',\
                                 larger_is_better = True,\
                                 cv = tscv5,\
                                 use_values = None,\
                                 use_recursion = False,\
                                 fill_na = FILL_NA,\
                                 show_progress = False)

In [ ]:
L2_scores = []
L2_features = []
for i in tqdm_notebook(range(1, 11)):  
    for _max_features in tqdm_notebook((100, 500, 1000)):
        D_datasets = _prepare_data(sites_full_str,\
                                   _X_ts_ohe_tr, _X_ts_ohe_te,\
                                   _X_ts_tr, _X_ts_te,\
                                   y_tr = y_tr,\
                                   vec = TfidfVectorizer(ngram_range = (1, i), max_features = _max_features))
        # отбираем признаки
        logit_selector.fit(D_datasets['logit_TR'], D_datasets['y_TR'])        
        x1_TR = logit_selector.transform(D_datasets['logit_TR'])
        x1_HOLD = logit_selector.transform(D_datasets['logit_HOLD'])
        x1_tr = logit_selector.transform(D_datasets['logit_tr'])
        x1_te = logit_selector.transform(D_datasets['logit_te'])

        # тестируем на отложенной
        logit_clf.fit(x1_TR, D_datasets['y_TR'])
        auc_logit_hold = roc_auc_score(D_datasets['y_HOLD'], logit_clf.predict_proba(x1_HOLD)[:, 1])

        # отбираем признаки
        lgb_selector.fit(D_datasets['lgb_TR'], D_datasets['y_TR'])        
        x2_TR = lgb_selector.transform(D_datasets['lgb_TR'])
        x2_HOLD = lgb_selector.transform(D_datasets['lgb_HOLD'])
        x2_tr = lgb_selector.transform(D_datasets['lgb_tr'])
        x2_te = lgb_selector.transform(D_datasets['lgb_te'])

        # тестируем на отложенной
        lgb_clf.fit(x2_TR, D_datasets['y_TR'])
        auc_lgb_hold = roc_auc_score(D_datasets['y_HOLD'], lgb_clf.predict_proba(x2_HOLD)[:, 1])        
        
        # собираем метрики, признаки
        L2_scores.append((logit_selector.best_score, auc_logit_hold, lgb_selector.best_score, auc_lgb_hold))
        L2_features.append(((x1_TR, x1_HOLD, x1_tr, x1_te),(x2_TR, x2_HOLD, x2_tr, x2_te)))    

<img src="стекинг_0402.png">

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklern.svm import LinearSVR¶
from sklearn.tree import DecisionTreeRegressor

# базовые модели
BASE_MODELS_LINEAR = [Lasso(random_state = SEED),\
                      Ridge(random_state = SEED),
                      KNeighborsRegressor(),\
                      LinearSVR(random_state = SEED)]
BASE_MODELS_TREE = [RandomForestRegressor(random_state = SEED),\
                    LGBMRegressor(random_state = SEED),\
                    XGBRegressor(random_state = SEED),\               
                    KNeighborsRegressor(),\
                    DecisionTreeRegressor(random_state = SEED)]
# итераций стекинга
N_ITERATIONS = 5
# число фолдов
N_FOLDS = 20
# доли признаков, на которых обучаются базовые модели
FEAT_USE_SHARE = .85
# итераций обучения на подпространствах признаков
N_SUBSET_ITERATIONS = 10

# признаки
L_logit_TR, L_logit_HOLD, L_logit_tr, L_logit_te = [], [], [], []
L_lgb_TR, L_lgb_HOLD, L_lgb_tr, L_lgb_te = [], [], [], []
for element in L2_features:
    L_logit_TR.append(element[0][0])
    L_logit_HOLD.append(element[0][1])
    L_logit_tr.append(element[0][2])
    L_logit_te.append(element[0][3])
    
    L_lgb_TR.append(element[1][0])
    L_lgb_HOLD.append(element[1][1])
    L_lgb_tr.append(element[1][2])
    L_lgb_te.append(element[1][3])
    
logits_L = (L_logit_TR, L_logit_HOLD, L_logit_tr, L_logit_te)
lgbs_L = (L_lgb_TR, L_lgb_HOLD, L_lgb_tr, L_lgb_te)

In [ ]:
LTR, LHOLD, Ltr, Lte = [], [], [], []

# запускаем стекинг N_ITERATIONS раз
for _ in tqdm_notebook(range(N_ITERATIONS)):    
    
    # используем признаки для logit и для lgb
    for feat_idx, selected_by in tqdm_notebook(enumerate((logits_L, lgbs_L))):
        
        # мы отбирали признаки 10 раз (для ngram_range от 1 до 10)
        for i in tqdm_notebook(range(len(selected_by))):
            
            # отобранные признаки
            subset = selected_by[i]
            _TR, _HOLD, _tr, _te = subset
            _y_TR, _y_HOLD = np.array(y_tr)[:IDX_SPLIT], np.array(y_tr)[IDX_SPLIT:]
            
            # обучаемся на разных подпространствах признаков
            for subset_idx in tqdm_notebook(range(1, N_SUBSET_ITERATIONS+1)):
                
                # фиксируем ансамбль
                # если используем признаки с ohe
                if feat_idx == 0:
                    # в качестве базовых моделей берем линейные
                    stacking_reg = StackingRegressor(models = BASE_MODELS_LINEAR,\
                                                     cv_strat = KFold(N_FOLDS, random_state = _))
                else:
                    # в качестве базовых моделей берем деревья
                    stacking_reg = StackingRegressor(models = BASE_MODELS_TREE,\
                                                     cv_strat = KFold(N_FOLDS, random_state = _))
                    
                
                # фиксируем сид
                np.random.seed(SEED+subset_idx)
                
                # выбираем признаки, которые будем использовать
                nfeatures = _TR.shape[1]
                n_feat_use = np.int32(np.around(nfeatures*FEAT_USE_SHARE))
                feat_subset = np.random.choice(np.arange(nfeatures), n_feat_use)
                
                # фитим ансамбль
                stacking_reg.fit(_TR[:, feat_subset], _y_TR)

                # получаем метапризнаки
                LTR.append(stacking_reg.get_metafeatures(_TR[:, feat_subset]))
                LHOLD.append(stacking_reg.get_metafeatures(_HOLD[:, feat_subset]))
                Ltr.append(stacking_reg.get_metafeatures(_tr[:, feat_subset]))
                Lte.append(stacking_reg.get_metafeatures(_te[:, feat_subset]))
